## Support Data Insight Analysis

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

## Loading Tasks and Agents YAML files

In [2]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents4.yaml',
    'tasks': 'config/tasks4.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [3]:
from crewai_tools import FileReadTool
csv_tool = FileReadTool(file_path='./support_tickets_data.csv')

In [4]:
# Creating Agents
suggestion_generation_agent = Agent(
  config=agents_config['suggestion_generation_agent'],
  tools=[csv_tool]
)

reporting_agent = Agent(
  config=agents_config['reporting_agent'],
  tools=[csv_tool]
)

chart_generation_agent = Agent(
  config=agents_config['chart_generation_agent'],
  allow_code_execution=True
)

# Creating Tasks
suggestion_generation = Task(
  config=tasks_config['suggestion_generation'],
  agent=suggestion_generation_agent
)

table_generation = Task(
  config=tasks_config['table_generation'],
  agent=reporting_agent
)

chart_generation = Task(
  config=tasks_config['chart_generation'],
  agent=chart_generation_agent
)

final_report_assembly = Task(
  config=tasks_config['final_report_assembly'],
  agent=reporting_agent,
  context=[suggestion_generation, table_generation, chart_generation]
)


# Creating Crew
support_report_crew = Crew(
  agents=[
    suggestion_generation_agent,
    reporting_agent,
    chart_generation_agent
  ],
  tasks=[
    suggestion_generation,
    table_generation,
    chart_generation,
    final_report_assembly
  ],
  verbose=True
)


In [ ]:
support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')

In [ ]:
support_report_crew.train(n_iterations=1, filename='training.pkl')

### Comparing new test results

In [ ]:
support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')

In [ ]:
# Display the Trello screenshot
from IPython.display import Image, display

# Load and display the image
test_image = Image(filename='test_before_training.png', width=368)
display(test_image)

In [7]:
result = support_report_crew.kickoff()

# Agent: Suggestion Engine
## Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use historical data such as resolution_time_minutes and
  satisfaction_rating to inform the suggestions.
- Customer Feedback: Incorporate insights from customer_comments to
  customize the suggestions further.

The goal is to provide clear, actionable steps that the support team can take to resolve each issue efficiently and effectively.



# Agent: Suggestion Engine
## Thought: I need to gather detailed information from the support tickets to generate actionable suggestions tailored to each issue reported.
## Using tool: Read a file's content
## Tool Input: 
"{}"
## Tool Output: 
ticket_id,customer_id,issue_type,issue_description,priority,date_submitted,response_time_minutes,resolution_time_minutes,satisfaction_rating,customer_comments,agent_id,re

KeyboardInterrupt: 

In [8]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

### Comprehensive Final Report

#### 1. Issue Classification Results
| Issue Type      | Frequency | Priority Levels (High/Medium/Low/Critical) |
|------------------|-----------|---------------------------------------------|
| API Issue        | 10        | High: 5, Low: 2, Critical: 3               |
| Login Issue      | 9         | High: 3, Low: 4, Critical: 2               |
| Report Generation | 7         | High: 3, Low: 3                             |
| Data Import      | 14        | High: 6, Medium: 5, Low: 3                  |
| Feature Request   | 8         | High: 5, Medium: 3                          |
| Billing Issue    | 13        | High: 6, Medium: 5, Low: 2, Critical: 3    |
| UI Bug            | 6         | High: 2, Medium: 3, Critical: 1            |

#### Issue Distribution Chart
![Issue Distribution](issue_distribution.png)

#### Priority Levels Chart
![Priority Levels](priority_levels.png)

---

#### 2. Agent Performance
| Agent ID | Total Tickets | Average Resolution Time (minutes) | Average Satisfaction Rating |
|----------|---------------|-----------------------------------|------------------------------|
| A001     | 9             | 897                               | 2.6                          |
| A002     | 10            | 780                               | 3.4                          |
| A003     | 12            | 688                               | 2.4                          |
| A004     | 12            | 693                               | 2.75                         |
| A005     | 7             | 783                               | 3.85                         |

#### Average Resolution Times per Agent Chart
![Average Resolution Time per Agent](resolution_times.png)

#### Agent Performance Chart
![Agent Performance](agent_performance.png)

---

#### 3. Customer Satisfaction Over Time
| Date       | Average Satisfaction Rating |
|------------|-----------------------------|
| Jan 2023   | 2.4                         |
| Feb 2023   | 2.5                         |
| Mar 2023   | 2.7                         |
| Apr 2023   | 2.6                         |
| May 2023   | 3.2                         |
| Jun 2023   | 3.1                         |
| Jul 2023   | 3.0                         |

#### Customer Satisfaction Over Time Chart
![Customer Satisfaction](customer_satisfaction.png)

---

#### 4. Suggested Actions
- **API Issues**: Conduct a review of the handling process and ensure responsiveness post-resolution.
- **Login Issues**: Investigate root causes promptly and establish a follow-up system, particularly for critical issues.
- **Report Generation**: Engage customers for specific improvements and emphasize follow-up practices.
- **Data Import**: Reinforce training on high-priority issue handling and address systemic challenges directly.
- **Feature Requests**: Improve follow-up communication processes and enhance collaboration for timely updates.
- **Billing Issues**: Facilitate immediate follow-up protocols, prioritize critical investigations, and open communication lines for resolution.
- **UI Bugs**: Implement feedback mechanisms for design improvement and enhance accountability in support responsiveness.

---

This report provides valuable insights into the performance of the support system and identifies key areas for improvement, allowing stakeholders to make informed decisions.